In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.var_model import VAR

In [2]:

df = pd.read_csv("D:/Masters/Research/DataCoSupplyChainDataset.csv", encoding='latin1')


In [3]:
# 
df = df.drop(columns=['shipping date (DateOrders)','Date1'])

# df = df.drop(columns=["Path"])



In [4]:
# df2=df.tail(1000)

In [5]:
df

,Days for shipping (real),Days for shipment (scheduled),Customer City,Customer Zipcode,Latitude,Longitude,Order City,Date,Time
0,5,4,Caguas,725.0,18.279451,-66.037064,Bikaner,18-01-2018,12:27
1,4,4,San Jose,95125.0,37.292233,-121.881279,Bikaner,17-01-2018,12:06
2,3,4,Los Angeles,90027.0,34.125946,-118.291016,Townsville,16-01-2018,11:45
3,2,4,Caguas,725.0,18.253769,-66.037048,Townsville,15-01-2018,11:24
4,6,4,Tonawanda,14150.0,43.013969,-78.879066,Toowoomba,19-01-2018,11:03
...,...,...,...,...,...,...,...,...,...
108958,4,4,Brooklyn,11207.0,40.640930,-73.942711,Shanghái,2016-01-20,03:40
108959,3,2,Bakersfield,93304.0,35.362545,-119.018700,Hirakata,2016-01-19,01:34
108960,5,4,Bristol,6010.0,41.629959,-72.967155,Adelaide,2016-01-20,21:00
108961,3,4,Caguas,725.0,18.213350,-66.370575,Adelaide,2016-01-18,20:18


In [6]:
df.to_csv("D:/Masters/Research/file.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108963 entries, 0 to 108962
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Days for shipping (real)       108963 non-null  int64  
 1   Days for shipment (scheduled)  108963 non-null  int64  
 2   Customer City                  108963 non-null  object 
 3   Customer Zipcode               108961 non-null  float64
 4   Latitude                       108963 non-null  float64
 5   Longitude                      108963 non-null  float64
 6   Order City                     108963 non-null  object 
 7   Date                           108963 non-null  object 
 8   Time                           108963 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 7.5+ MB


In [8]:
# df=df2

In [9]:
import pandas as pd
import networkx as nx
from geopy.distance import geodesic
from opencage.geocoder import OpenCageGeocode

In [10]:
OPENCAGE_API_KEY = '53cea7afa5ea42a99f235b750b22a19d'

In [11]:
geocoder = OpenCageGeocode(OPENCAGE_API_KEY)

In [12]:
def get_coordinates(city):
    result = geocoder.geocode(city)
    if result and len(result):
        return result[0]['geometry']['lat'], result[0]['geometry']['lng']
    else:
        return None, None

In [13]:
unique_cities = pd.concat([df['Order City'], df['Customer City']]).unique()
unique_cities

array(['Bikaner', 'Townsville', 'Toowoomba', ..., 'Morganton', 'CA',
       'Sumner'], dtype=object)

In [ ]:
# Create a dictionary to store coordinates of cities
city_coords = {}
for city in unique_cities:
    lat, lon = get_coordinates(city)
    if lat is not None and lon is not None:
        city_coords[city] = (lat, lon)

In [ ]:
city_coords

In [ ]:
cords=pd.DataFrame(city_coords).T
cords["Place"]=cords.index
cords.to_csv("D:/Masters/Research/cords.csv")


In [ ]:
cords=pd.read_csv("D:/Masters/Research/cords.csv")

In [ ]:
# Create a list of city names and their coordinates
city_names = list(city_coords.keys())
coords = np.array(list(city_coords.values()))

In [ ]:
import pandas as pd
from geopy.distance import geodesic
import networkx as nx
from sklearn.neighbors import NearestNeighbors
import numpy as np
# Use NearestNeighbors to find the nearest N cities for each city
N = 5  # Number of nearest neighbors
nbrs = NearestNeighbors(n_neighbors=N, algorithm='ball_tree').fit(coords)
distances, indices = nbrs.kneighbors(coords)

In [ ]:
# Create a graph of cities with distances as weights
G = nx.Graph()
for i, city in enumerate(city_names):
    for j in range(1, N):  # Skip the first one because it's the city itself
        neighbor_city = city_names[indices[i][j]]
        distance = geodesic(city_coords[city], city_coords[neighbor_city]).miles
        G.add_edge(city, neighbor_city, weight=distance)

In [ ]:
# Function to find the shortest path of cities
def find_path(order_city, customer_city):
    if order_city not in city_coords or customer_city not in city_coords:
        return None
    try:
        path = nx.shortest_path(G, source=order_city, target=customer_city, weight='weight')
        return ' -> '.join(path)
    except nx.NetworkXNoPath:
        return None

In [ ]:
# Add the path to the dataset
df['Path'] = df.apply(lambda row: find_path(row['Order City'], row['Customer City']), axis=1)



In [ ]:
df.info()

In [ ]:
df2= df.dropna()

In [ ]:
df2.Path.iloc[1]

In [ ]:
df2.info()

In [ ]:

# Split the Path column into separate columns
path_columns = df2['Path'].str.split(' -> ', expand=True)


path_columns

In [ ]:
# Assuming df2 is your main DataFrame and path_columns is the DataFrame you want to save inside df2
df2["Path2"] = [pd.DataFrame(path_columns.iloc[i].dropna()) for i in range(len(path_columns))]


In [ ]:
df2

In [ ]:
df2.Path2.iloc[1].iloc[1]

In [ ]:
import requests

url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Miami/2015-01-11/2015-01-16?key=XER7B2RE72KSTRX2DXA2867WF"
response = requests.get(url)

# Check if the response was successful
if response.status_code == 200:
    # Print the response content (assuming it is JSON)
    try:
        data = response.json()  # Parse the response as JSON
        print(data)  # Print the parsed data
    except ValueError:
        print("Error: Response content is not in JSON format")
else:
    print(f"Error: Received response with status code {response.status_code}")


In [ ]:
# fetch_weather_data("Cork","2015-01-16","2015-01-17")
fetch_weather_data(df2['Path2'].iloc[1].iloc[2].item(), histo_dat[1]['Start Date'], histo_dat[1]['End Date'])
# l=df2['Path2'].iloc[1].iloc[1]
# k=l.item()
# k

# df2['Path2'].iloc[1].iloc[2].item()

In [ ]:
df2['Path2'].iloc[1].iloc[2], histo_dat[1]['Start Date'], histo_dat[1]['End Date']

In [ ]:
# Define function to fetch weather data
def fetch_weather_data(city, start_date, end_date):
    api_key = "FBEMAY2RDDD3646FG2AXZ55WV"
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{start_date}/{end_date}?key={api_key}&include=days"
    response = requests.get(url)
    # Check if the response was successful
    if response.status_code == 200:
    # Print the response content (assuming it is JSON)
        try:
            data = response.json()  # Parse the response as JSON
            return(pd.DataFrame(data["days"]))  # Print the parsed data
        except ValueError:
            print("Error: Response content is not in JSON format")
    else:
        print(f"Error: Received response with status code {response.status_code}")
        

In [ ]:
from datetime import datetime, timedelta
# Prepare date ranges and fetch weather data
histo_dat = []
for index, row in df2.iterrows():
    date_str = row['Date']
    try:
        date_obj = datetime.strptime(date_str, '%d-%m-%Y')
        start_date = (date_obj - timedelta(days=5)).strftime('%Y-%m-%d')
        end_date = date_obj.strftime('%Y-%m-%d')
        histo_dat.append({
                'Start Date': start_date,
                'End Date': end_date,
            })
    except ValueError:
        # Skip rows with invalid date format
        continue


            

In [ ]:
from datetime import datetime, timedelta
# Prepare date ranges and fetch weather data
forecast_date = []
for index, row in df2.iterrows():
    date_str = row['Date']
    try:
        date_obj = datetime.strptime(date_str, '%d-%m-%Y')
        start_date = date_obj.strftime('%Y-%m-%d')
        end_date = (date_obj + timedelta(days=row["Days for shipping (real)"])).strftime('%Y-%m-%d')
        forecast_date.append({
                'Start Date': start_date,
                'End Date': end_date,
            })
    except ValueError:
        # Skip rows with invalid date format
        continue


            

In [ ]:
# histo_dat

In [ ]:
# forecast_date

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
import pandas as pd

def forecast_weather(city, start_date_hist, end_date_hist, start_date_forecast, end_date_forecast):
    # Fetch historical weather data
    df_hist = fetch_weather_data(city, start_date_hist, end_date_hist)
    
    if df_hist is None:
        print("Failed to fetch historical weather data")
        return None

    # Filter out constant columns from df_hist
    df_hist_filtered = df_hist.loc[:, (df_hist != df_hist.iloc[0]).any()]

    # Ensure numeric data types
    df_hist_filtered = df_hist_filtered.apply(pd.to_numeric, errors='coerce')

    # Check for NaN values and handle them appropriately (fill with mean, median, or dropna)
    df_hist_filtered = df_hist_filtered.dropna()

    # Check if there's sufficient data
    if len(df_hist_filtered) < 5:  # Adjust this number based on your data requirements
        print("Insufficient data for VAR model fitting")
        return df_hist_filtered

    # Fit the VAR model
    try:
        model = VAR(df_hist_filtered)
        model_fit = model.fit(maxlags=3)  # Adjust maxlags as needed

        # Forecast
        forecast_steps = (end_date_forecast - start_date_forecast).days + 1
        forecast = model_fit.forecast(df_hist_filtered.values[-model_fit.k_ar:], steps=forecast_steps)

        # Create a DataFrame for the forecast
        forecast_df = pd.DataFrame(forecast, index=pd.date_range(start=start_date_forecast, periods=forecast_steps, closed='right'), columns=df_hist_filtered.columns)

        # Print the forecast (optional)
        print(forecast_df)

        return forecast_df

    except Exception as e:
        print(f"Error fitting VAR model: {str(e)}")
        return None

# Example usage
forecast_weather("cork", "2015-05-10", "2015-06-10", "2015-06-11", "2015-06-12")


In [ ]:
l=fetch_weather_data("cork", "2015-06-05", "2015-06-10")
df_hist_filtered = l.loc[:, (l != l.iloc[0]).any()]
# df_hist_filtered = df_hist_filtered.apply(pd.to_numeric, errors='coerce')
# df_hist_filtered = df_hist_filtered.dropna()
# df_hist_filtered.isna().sum()
df_hist_filtered.info()

In [ ]:
import requests
# Collect results in a list
results = []

# Assuming df2, histo_dat, and forecast_date are defined and structured appropriately
for i, path in enumerate(df2['Path2']):
    city_forecast = {}
    for city in path:
        forecast_df = forecast_weather(city, histo_dat[i]['Start Date'], histo_dat[i]['End Date'], forecast_date[i]['Start Date'], forecast_date[i]['End Date'])
        if forecast_df is not None:
            city_forecast[city] = forecast_df
    results.append(city_forecast)

In [ ]:
df2.Path2.iloc[1]

In [ ]:
histo_dat[1]['Start Date'], histo_dat[1]['End Date'], forecast_date[1]['Start Date'], forecast_date[1]['End Date']

In [ ]:
df2['Path2'].iloc[1].iloc[1]

In [ ]:
# Convert to DataFrame
dtta = pd.DataFrame(k['days'][0]['hours'])

dtta

In [ ]:
dta